# Importing the required python libraries



In [1]:

import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt 
import seaborn as sb # used to plot the heatmap
import csv

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error,  mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(0)
start = time.time()

# Creating a python class for loading the data into Neural network

In [2]:
class TabularDataset(Dataset):
    def __init__(self, data, cat_cols=None, output_col=None):
        """
        Characterizes a Dataset for PyTorch

        Parameters
        ----------

        data: pandas data frame
        The data frame object for the input data. It must
        contain all the continuous, categorical and the
        output columns to be used.

        cat_cols: List of strings
        The names of the categorical columns in the data.
        These columns will be passed through the embedding
        layers in the model. These columns must be
        label encoded beforehand. 

        output_col: string
        The name of the output variable column in the data
        provided.
        """

        self.n = data.shape[0] # n stores the total number of data points

        if output_col:
            self.y = data[output_col].astype(np.float32).values.reshape(-1, 1) #data[output_col] contains two columns 
                                                                            #(first column is the index). This line converts it 
                                                                            # to single column
            
        else:
            self.y = np.zeros((self.n, 1)) # a general case when out

        self.cat_cols = cat_cols if cat_cols else [] # categorical inputs are stored in "self.cat_cols"
        self.cont_cols = [
            col for col in data.columns if col not in self.cat_cols + [output_col] # finds out the columns that have 
                                                                                   # contionuos inputs
        ]

        if self.cont_cols:
            self.cont_X = data[self.cont_cols].astype(np.float32).values # The data corresponding to 
                                                                         # continuous columns are stored in "self.cont_x"
        else:
            self.cont_X = np.zeros((self.n, 1))

        if self.cat_cols:
            self.cat_X = data[cat_cols].astype(np.int64).values # The data corresponding to 
                                                                # categorical columns are stored in "self.cont_x"
        else:
            self.cat_X = np.zeros((self.n, 1))

    def __len__(self):
        """
        Denotes the total number of samples.
        """
        return self.n

    def __getitem__(self, idx):
        """
        Generates one sample of data as a triplet --> #(output, continous inputs, categorical inputs)
        """
        return [self.y[idx], self.cont_X[idx], self.cat_X[idx]]

# Definition of the Neural network

In [3]:
class FeedForwardNN(nn.Module):
    def __init__(
        self,
        emb_dims,
        no_of_cont,
        lin_layer_sizes,
        output_size,
        emb_dropout,
        lin_layer_dropouts,
    ):

        """
        Parameters
        ----------

        emb_dims: List of two element tuples
        This list will contain a two element tuple for each
        categorical feature. The first element of a tuple will
        denote the number of unique values of the categorical
        feature. The second element will denote the embedding
        dimension to be used for that feature.

        no_of_cont: Integer
        The number of continuous features in the data.

        lin_layer_sizes: List of integers.
        The size of each linear layer. The length will be equal
        to the total number
        of linear layers in the network.

        output_size: Integer
        The size of the final output.

        emb_dropout: Float
        The dropout to be used after the embedding layers.

        lin_layer_dropouts: List of floats
        The dropouts to be used after each linear layer.
        """

        super().__init__()

        # Embedding layers of categorical inputs
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])

        no_of_embs = sum([y for x, y in emb_dims])
        self.no_of_embs = no_of_embs
        self.no_of_cont = no_of_cont

        # Linear Layers of the nueral network
        first_lin_layer = nn.Linear(
            self.no_of_embs + self.no_of_cont, lin_layer_sizes[0]
        ) # first layer where data is fed into the network
        # Following lines define the hidden layers
        self.lin_layers = nn.ModuleList(
            [first_lin_layer]
            + [
                nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1])
                for i in range(len(lin_layer_sizes) - 1)
            ]
        )
        # Initializing the weights of neural network layers
        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)

        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1], output_size)
        nn.init.kaiming_normal_(self.output_layer.weight.data)

        # Batch Norm Layers, It is used to normalize the data into the layers to avoid overfitting and to optimize network
        self.first_bn_layer = nn.BatchNorm1d(self.no_of_cont)
        self.bn_layers = nn.ModuleList(
            [nn.BatchNorm1d(size) for size in lin_layer_sizes]
        )

        # Dropout Layers used as regularization
        self.emb_dropout_layer = nn.Dropout(emb_dropout)
        self.droput_layers = nn.ModuleList(
            [nn.Dropout(size) for size in lin_layer_dropouts]
        )

    def forward(self, cont_data, cat_data):
        # This function defines the mathematical operations of the neural network 
        if self.no_of_embs != 0: # Defines the embedding layer of categorical inputs
            x = [
                emb_layer(cat_data[:, i]) for i, emb_layer in enumerate(self.emb_layers)
            ]
            x = torch.cat(x, 1)
            x = self.emb_dropout_layer(x) # dropout regularization of categorical inputs 

        if self.no_of_cont != 0:
            normalized_cont_data = self.first_bn_layer(cont_data) # defines the linear (normal nueral net hidden layer)
                                                                  # for continous inputs

            if self.no_of_embs != 0:
                x = torch.cat([x, normalized_cont_data], 1)
            else:
                x = normalized_cont_data
        # Batch normalization operation is done in this for loop
        for lin_layer, dropout_layer, bn_layer in zip( 
            self.lin_layers, self.droput_layers, self.bn_layers
        ):

            x = F.relu(lin_layer(x))
#             x = F.sigmoid(lin_layer(x))
            x = bn_layer(x)
            x = dropout_layer(x)

        x = self.output_layer(x)

        return x # Returns the output of the network

# Data loading and preprocessing

In [4]:
# The data is read as a pandas data frame
data = pd.read_csv("finalized_data.csv").dropna()

In [5]:
data.columns


Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], dtype='object')

### One-hot encoding of categorical features

In [6]:
# Array that specifies which columns are categorical inputs
categorical_features = ["0", "1", "2", "3", "4","5", "6", "7", "8"] # Except column "7", all are label encoded and "7" is 
                                                               # one-hot encoded. 
# Array that specifies which columns are continuous inputs
contnuous_features = ["9","10","11"]
output_feature = "12"  # specifies which column is output


temp={}
label_encoders = {} 
# for loop starts for label encoding
for i in range(0,len(categorical_features)):
    #codes for one-hot encoding
    label_encoders[categorical_features[i]] = OneHotEncoder(handle_unknown='ignore')
    label_encoders[categorical_features[i]].fit(data[categorical_features[i]].values.reshape(-1,1))
    temp[categorical_features[i]]=label_encoders[categorical_features[i]].transform(data[categorical_features[i]].values.reshape(-1,1)).toarray() 
    

In [7]:
#printing the dimensions of each categorical variables after one-hot encoding
for i in range(0,len(temp)):
    print(temp[categorical_features[i]].shape)

(1263600, 13)
(1263600, 3)
(1263600, 3)
(1263600, 3)
(1263600, 3)
(1263600, 4)
(1263600, 3)
(1263600, 4)
(1263600, 21)


In [8]:
# Adding the one hot encoded features into the data
# The new columns will be named from 13 onwards.
ind=13
new_categorical_features=[]
for i in range(0,len(categorical_features)):
    for j in range(0,temp[categorical_features[i]].shape[1]):
        data[str(ind)] = temp[categorical_features[i]][:,j]
        new_categorical_features.append(str(ind)) # Stroing the column identity of newly added one-hot encoded columns as 
                                                  # categorical input.
        ind=ind+1
    data=data.drop([categorical_features[i]], axis=1) # The old column is deleted from the data
      

In [9]:
new_categorical_features

['13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69']

In [10]:
print(data) # Data after one-hot encoding.
print(data.shape)     # New dimension of the data. 

                 9         10         11        12   13   14   15   16   17  \
0        -1.000000 -13.016747   6.399998  0.010556  1.0  0.0  0.0  0.0  0.0   
1        -1.000000 -13.016747   7.199998  0.011241  1.0  0.0  0.0  0.0  0.0   
2        -1.000000 -13.016747   7.999998  0.012185  1.0  0.0  0.0  0.0  0.0   
3        -1.000000 -13.016747   8.799997  0.013634  1.0  0.0  0.0  0.0  0.0   
4        -1.000000 -13.016747   9.599998  0.014240  1.0  0.0  0.0  0.0  0.0   
...            ...        ...        ...       ...  ...  ...  ...  ...  ...   
1263595  12.283293  -3.000003  66.399963  0.000000  0.0  0.0  0.0  0.0  0.0   
1263596  12.283293  -3.000003  67.199959  0.000000  0.0  0.0  0.0  0.0  0.0   
1263597  12.283293  -3.000003  67.999962  0.000000  0.0  0.0  0.0  0.0  0.0   
1263598  12.283293  -3.000003  68.799957  0.000000  0.0  0.0  0.0  0.0  0.0   
1263599  12.283293  -3.000003  69.599960  0.000000  0.0  0.0  0.0  0.0  0.0   

          18  ...   60   61   62   63   64   65   6

In [11]:
data.columns

Index(['9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69'],
      dtype='object')

In [12]:
filename = 'index.csv'
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)
index = np.array(x).astype('float')

In [ ]:
X=data.to_numpy() 
instances=X.shape[0]/25
# index = np.random.permutation(int(instances))
train=np.zeros((int(np.floor(0.8*instances))*25, X.shape[1]))
print(train.shape)
 

for i in range(int(np.floor(0.8*instances))):
    t1=i*25
    t2=int(index[i]*25)
    train[t1:t1+25,:] = X[t2:t2+25,:]
 

print(train.shape)
t4=int(instances)-int(np.floor(0.8*instances))
test_new=np.zeros((t4*25, data.shape[1]))
 
t3=int(np.floor(0.8*instances))

for i in range(t4):
    t1=i*25
    t2=int(index[i+t3]*25)
    test_new[t1:t1+25,:] = X[t2:t2+25,:]
 
print(test_new.shape)

In [ ]:
# train_permuted=np.zeros((train.shape[0], train.shape[1]))

# instances1=train.shape[0]
# index1 = np.random.permutation(int(instances1))
# for i in range(instances1):
#     train_permuted[i,:]=train[int(index1[i]),:]
    
    

In [ ]:
# Splitting the data into training (80%) and testing (20%)
# train_all, test = train_test_split(data, test_size=0.2, shuffle=True) 
# Splitting the training data into training set (7/8 portion) into training set and validation set (1/8 portion)
val, test = train_test_split(test_new, test_size=2/3, shuffle = False)

In [ ]:
print(train.shape)
# print(val.shape)
print(test_new.shape)

In [ ]:
def NN_model(arch, emb_dims, dropout_rate, no_of_epochs, criterion, lr, traindataloader, valdataloader, testdataloader, ground_label, is_test):
    device = torch.device('cuda')
    model = FeedForwardNN(emb_dims, no_of_cont=3, lin_layer_sizes=arch, output_size=1, emb_dropout=dropout_rate, 
                          lin_layer_dropouts=[dropout_rate] * len(arch)).to(device)
    
    no_of_epochs = no_of_epochs # Number of times the data is fed into the network 
    criterion = criterion # Objective function of the network
    lr=lr
    optimizer = torch.optim.Adam(model.parameters(), lr) # Optimization algorithm to be used
    stopper=10 # A value to check overfitting
    loss_train_prev=0 # VAriable to store value of objective function in the previous epoch
    Ltrain=[] # Python list to store training loss in each epoch
    Ltest=[] # Python list to store testing loss in each epoch
    
    
    # Neural network training starts from here
    for epoch in range(no_of_epochs):
      loss_var=[]  
      for y, cont_x, cat_x in traindataloader: # Loading the training dataset
          
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)

        # Forward Pass
        preds = model(cont_x, cat_x)
        loss = criterion(preds, y)
        loss_var.append(loss.item())
#         print(loss_var)
#         print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        
        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#       print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
      loss_train=sum(loss_var)/(train.shape[0]/batchsize_train) # Finding out the objective function
       
    
      Ltrain.append(loss_train)
      
       
    
      # Doing validation of the Neural network
      loss_var=[]  
      for y, cont_x, cat_x in valdataloader:
        
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)

        # Forward Pass
        preds = model(cont_x, cat_x)
        loss = criterion(preds, y)
        loss_var.append(loss.item())
      loss_test=sum(loss_var)/(val.shape[0]/batchsize_val)  # Finding out the objective function
       
      Ltest.append(loss_test)
#         print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        
        # Backward Pass and Optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
      if is_test==False:
          if epoch % 20==0:
              print('Epoch {}: train loss: {} validation loss: {}'.format(epoch, loss_train, loss_test))
      else:
          print('Epoch {}: train loss: {} validation loss: {}'.format(epoch, loss_train, loss_test))
              
      if epoch > stopper and Ltest[-1] > np.mean(Ltest[-(stopper+1):-1]):  # Checking overfitting 
            print("Early Stoppin.....")
            break
            
    # Testing
    predicted=np.zeros((0,1))  # Variable to store neural network prediction
    for y, cont_x, cat_x in testdataloader:
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
#         y  = y.to(device)
        preds = model(cont_x, cat_x)
        preds=preds.cpu()
        predicted = np.concatenate((predicted, preds.detach().numpy()),axis=0)
    actual = ground_label # Variable storing the actual values of the testing cases
    actual=actual.reshape(actual.shape[0],1) # Reshaping for visualization
    result=np.concatenate((actual, predicted),axis=1) # Actual value and predicted value is stored in "result" variable.
    
    MSE = mean_squared_error(actual, predicted)
    MAE = mean_absolute_error(actual, predicted)
    R = r2_score(actual, predicted)
    
    if is_test:
        np.savetxt('NN_result.csv', result, delimiter=',')
    print('MSE: {} MAE: {} R2: {}'.format(MSE, MAE, R))
    return MSE, MAE, R, predicted

In [ ]:
n_splits = 5
kf = KFold(n_splits)
kf.get_n_splits(train) 
KFold(n_splits=5, random_state=None, shuffle=False)
architectures=[ [64], [128], [256], [512], [1024], [64,64], [128,128], [256,256], [512,512], [1024,1024], 
               [64,64,64], [128,128,128], [256,256,256], [512,512,512], [1024,1024,1024], 
              [64,64,64,64], [128,128,128,128], [256,256,256,256], [512,512,512,512], [1024,1024,1024,1024],
              ]
Results = np.zeros((len(architectures), n_splits,3))
cat_dims = [2 for col in new_categorical_features]
    
# Finding out the embedding dimensions for the categorical features
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
dropout_rate = 0.5
no_of_epochs = 100
criterion = nn.MSELoss()
lr = 0.01
k=0
for arch in architectures:
    print("-------------------------------------------------------")
    print(arch)
    print("-------------------------------------------------------")
    p=0
    for train_index, test_index in kf.split(train):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = train[train_index], train[test_index]
        val_k, test_k = train_test_split(X_test, test_size=2/3, shuffle = False)
    
        train_k=pd.DataFrame(X_train)
        val_k=pd.DataFrame(val_k)
        test_k=pd.DataFrame(test_k)

        train_columns=[]
        for i in range(0,61):
            train_columns.append(str(i))
        train_k.columns=train_columns
        test_k.columns=train_columns
        val_k.columns=train_columns

        new_categorical_features=train_columns[4:]
        output_feature=train_columns[3]
     
        # Creating the training dataset to feed into the network
        traindataset = TabularDataset(data=train_k, cat_cols=new_categorical_features,
                             output_col=output_feature)
        # Creating the validation dataset to feed into the network
        valdataset = TabularDataset(data=val_k, cat_cols=new_categorical_features,
                             output_col=output_feature)
        # Creating the testing dataset to feed into the network
        testdataset = TabularDataset(data=test_k, cat_cols=new_categorical_features,
                             output_col=output_feature)
    
        batchsize_train = 16000
        batchsize_val=1381
        batchsize_test=42121
        # Defining pytorch dataloader for train, validation and test dataset
        traindataloader = DataLoader(traindataset, batchsize_train, shuffle=True, num_workers=0, ) 
        valdataloader = DataLoader(valdataset, batchsize_val, shuffle=False, num_workers=0)
        testdataloader = DataLoader(testdataset, batchsize_test, shuffle=False, num_workers=0)
    
        # Finding out the uniue values corresponding to the categoricl features
        
        MSE, MAE, R, predicted = NN_model(arch, emb_dims, dropout_rate, no_of_epochs, criterion, lr, traindataloader, 
                               valdataloader, testdataloader, test_k['3'].to_numpy(), is_test = False)
         #arch, emb_dims, dropout_rate, no_of_epochs, criterion, lr, traindataloader, valdataloader, testdataloader, 
            #ground_label, is_test):
   
        Results[k,p,0] = MSE
        Results[k,p,1] = MAE
        Results[k,p,2] = R
        p+=1
    k+=1

# Neural network training

In [ ]:
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")

In [ ]:
Results.shape

In [ ]:
np.savetxt('Cross_validation_MSE_results.csv', Results[:,:,0], delimiter=',')
np.savetxt('Cross_validation_MAE_results.csv', Results[:,:,1], delimiter=',')
np.savetxt('Cross_validation_R2_results.csv', Results[:,:,2], delimiter=',')
RMSE_summary = Results[:,:,0]
RMSE_summary = np.sum(RMSE_summary, axis = 1)
RMSE_summary = np.argmin(RMSE_summary)
 
arch = architectures[RMSE_summary]
print("Architecture selected through cross validation:")
print(arch)
train=pd.DataFrame(train)
val=pd.DataFrame(val)
test=pd.DataFrame(test)

train_columns=[]
for i in range(0,61):
    train_columns.append(str(i))
train.columns=train_columns
test.columns=train_columns
val.columns=train_columns

new_categorical_features=train_columns[4:]
output_feature=train_columns[3]
     
# Creating the training dataset to feed into the network
traindataset = TabularDataset(data=train, cat_cols=new_categorical_features,
                             output_col=output_feature)
# Creating the validation dataset to feed into the network
valdataset = TabularDataset(data=val, cat_cols=new_categorical_features,
                             output_col=output_feature)
# Creating the testing dataset to feed into the network
testdataset = TabularDataset(data=test, cat_cols=new_categorical_features,
                             output_col=output_feature)
    
batchsize_train = 16000
batchsize_val=1381
batchsize_test=42121
# Defining pytorch dataloader for train, validation and test dataset
traindataloader = DataLoader(traindataset, batchsize_train, shuffle=True, num_workers=0) 
valdataloader = DataLoader(valdataset, batchsize_val, shuffle=False, num_workers=0)
testdataloader = DataLoader(testdataset, batchsize_test, shuffle=False, num_workers=0)
    
# Finding out the uniue values corresponding to the categoricl features
cat_dims = [2 for col in new_categorical_features]
    
# Finding out the embedding dimensions for the categorical features
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
dropout_rate = 0.5
no_of_epochs = 100
criterion = nn.MSELoss()
lr = 0.01
actual= test['3'].to_numpy()
MSE, MAE, R, predicted = NN_model(arch, emb_dims, dropout_rate, no_of_epochs, criterion, lr, traindataloader, valdataloader, testdataloader, actual,is_test = True)
         

In [ ]:
# The mean suared error of the prediction is printed

print("Mean squared error is:")
print(MSE)
print("Mean absolute error is:")
print(MAE)
print("R2 score is:")
print(R)

# Output visualization

In [ ]:
actual =  test['3'].to_numpy()
actual = np.reshape(actual,(actual.shape[0],1))
real=np.zeros((25,1))
predctd = np.zeros((25,1))

pos = int(input("Enter an integer between 0 to 6738 to test the model : "))
pos = pos*25+9

real=actual[pos:pos+25,-1]
predctd = predicted[pos:pos+25,-1]

real=np.reshape(real,(5,5))
predctd = np.reshape(predctd,(5,5))


print("Heatmap Actual and Heatmap predicted")
plt.figure(figsize=(12,4))
plt.subplot(121)
heat_map = sb.heatmap(real, vmin = 0, vmax = 1.30141, cmap="OrRd_r", annot=True)
plt.subplot(122)
heat_map = sb.heatmap(predctd,  vmin = 0, vmax = 1.30141, cmap="OrRd_r", annot=True)